In [129]:
import warnings
warnings.filterwarnings('ignore')

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [7]:
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess

In [8]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [12]:
yelp['text'].apply(lambda x: tokenize(x)).head()

0    [beware, fake, fake, fake, small, business, lo...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ve, vegas, dozens, times, stepped, foot, circ...
3    [went, night, closed, street, party, best, act...
4    [stars, bad, price, lunch, seniors, pay, eatin...
Name: text, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
pd.options.display.max_columns = None

In [56]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit_transform(yelp.text)
dtm = vectorizer.transform(yelp.text)

In [53]:
dtm.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [71]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [77]:
fake_review = ["""I think this is the worst place ever, I don't like this cafe. This is the worst!"""]

In [78]:
new = vectorizer.transform(fake_review)

In [79]:
nn.kneighbors(new.todense())

(array([[3.16227766, 3.31662479, 3.31662479, 3.46410162, 3.46410162,
         3.46410162, 3.46410162, 3.46410162, 3.46410162, 3.46410162]]),
 array([[5129, 9020, 3543, 9509, 2515, 5087, 2401, 8023, 6875, 6847]]))

In [85]:
yelp.iloc[5129].text + '\n', yelp.iloc[9020].text + '\n', yelp.iloc[3543].text + '\n', yelp.iloc[9509].text + '\n', yelp.iloc[2515].text + '\n', yelp.iloc[5087].text + '\n', yelp.iloc[2401].text + '\n', yelp.iloc[8023].text + '\n', yelp.iloc[6875].text + '\n', yelp.iloc[6847].text + '\n'

('Best place everrrrr!!!\n',
 'I decided to try this place. Everyone was friendly however I would never go back.  I got the worst fungus in my fingernail.\n',
 '$25 for 2 weeks\n',
 'worst place do not waste your time here!\nLas Vegas has many options dont pick this now!\n',
 'Not huge, but an interesting collection.\n',
 'best place in Pittsburgh wish it was bigger.\n',
 "SUPER SLOW!!!! If you are in any kind of hurry don't go here.\n",
 'Kids fav place! Swedish pancakes!!\n',
 'The turn around date is great.\n',
 'Just because I was in the area and I saw the show, worst experience and crap food.\n')

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [87]:
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
vect = TfidfVectorizer(stop_words='english')
sgdc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

In [93]:
pipe.fit(yelp.text, yelp.stars)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [97]:
pipe.predict(fake_review)

array([1])

In [98]:
from sklearn.model_selection import GridSearchCV

In [102]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'clf__max_iter': (20, 10, 100)
}

In [103]:
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)

In [104]:
grid_search.fit(yelp.text, yelp.stars)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  4.3min finished
/Users/dwightchurchill/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'clf__max_iter': (20, 10, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [125]:
import re
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [138]:
id2word = corpora.Dictionary(yelp.text.apply(lambda x: tokenize(x)))

In [139]:
id2word.token2id['bar']

137

In [140]:
corpus = [id2word.doc2bow(text) for text in yelp.text.apply(lambda x: tokenize(x))]

In [122]:
lda = LdaMulticore(corpus=corpus,
                  id2word=id2word,
                  random_state=723812,
                  num_topics=15, 
                  passes=10,
                  workers=4
                  )

In [123]:
lda.print_topics()

[(0,
  '0.029*"room" + 0.012*"hotel" + 0.009*"got" + 0.006*"service" + 0.005*"rooms" + 0.005*"like" + 0.005*"stay" + 0.005*"want" + 0.005*"check" + 0.005*"nice"'),
 (1,
  '0.011*"said" + 0.010*"like" + 0.007*"time" + 0.007*"car" + 0.007*"told" + 0.007*"day" + 0.005*"got" + 0.005*"called" + 0.005*"people" + 0.005*"asked"'),
 (2,
  '0.011*"staff" + 0.010*"recommend" + 0.010*"time" + 0.009*"great" + 0.009*"dr" + 0.008*"friendly" + 0.008*"service" + 0.007*"nails" + 0.007*"office" + 0.007*"nail"'),
 (3,
  '0.014*"great" + 0.010*"food" + 0.009*"service" + 0.009*"good" + 0.009*"time" + 0.008*"place" + 0.006*"like" + 0.006*"best" + 0.005*"friendly" + 0.004*"ice"'),
 (4,
  '0.021*"food" + 0.014*"chicken" + 0.013*"good" + 0.009*"ordered" + 0.009*"service" + 0.007*"like" + 0.007*"restaurant" + 0.007*"place" + 0.006*"sauce" + 0.006*"menu"'),
 (5,
  '0.011*"great" + 0.007*"nice" + 0.006*"pool" + 0.006*"little" + 0.005*"like" + 0.005*"area" + 0.005*"happy" + 0.004*"time" + 0.004*"went" + 0.004*"staf

In [126]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [127]:
topics = [' '.join(t[0:5]) for t in words]

In [128]:
for t in topics: 
    print(t)
    print("\n")

room hotel got service rooms


said like time car told


staff recommend time great dr


great food service good time


food chicken good ordered service


great nice pool little like


great place food good time


place good pizza food great


food good place great service


hair great ve time store


like place great experience best


great place restaurant way lot


service time work order great


service new time good car


place food good great service




In [141]:
distro = [lda[d] for d in corpus]

In [142]:
distro[0]

[(1, 0.500314),
 (9, 0.03369894),
 (12, 0.24720155),
 (13, 0.05526729),
 (14, 0.14836664)]

In [143]:
def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [144]:
d_dist = {k:0 for k in range(0,15)}

In [148]:
df = pd.DataFrame.from_records(new_distro)
df.columns = topics

In [ ]:
df.head(10)

In [166]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

### Analysis of results


## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)